<a href="https://akademie.datamics.com/kursliste/">![title](bg_datamics_top.png)</a>

<center><em>© Datamics</em></center><br><center><em>Besuche uns für mehr Informationen auf <a href='https://akademie.datamics.com/kursliste/'>www.akademie.datamics.com</a></em>

# Optischer Fluss (Optical Flow)

----
#### BEACHTE: Es ist wahrscheinlich eine gute Idee, den Kernel neu zu starten, wenn du diese Zellen jemals ausführen solltest, da der Tracking algorithm (Folgealgorithmus) manchmal in einer Schleife mit der Kamera hängenbleibt.

## Lucas-Kanade Optischer Fluss

In [ ]:
import numpy as np
import cv2 

In [ ]:
# Parameter für ShiTomasi-Corner Detection (Eckenerkennung) (Paper Good Features to Track)
corner_track_params = dict(maxCorners = 10,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

### Parameter für den Lucas Kanade Optischen Fluss

Erkenne die Bewegung spezifischer Punkte oder die aggregierte Bewegung ganzer Bereiche durch Anpassung des winSize-Argumentes. Dieses Argument bestimmt die Grösse des Integrationsfensters (Integration Window). Kleine Fenster sind anfälliger für Störsignale und können größere Bewegungen übersehen. Große Fenster "überleben" einen Einschluss (occlusion).

Die Integration erscheint glatter mit einer größeren Fenstergröße.

criteria hat hier zwei Werte - die maximale Anzahl von Iterationen (unter 10) und Epsilon (unter 0.03). Mehr Iterationen bedeuten eine erschöpfende Suche und ein kleineres Epsilon terminiert früher. Diese Argumente sind hauptsächlich nützlich beim Abwägen von Geschwindigkeit gegen Genauigkeit, bleiben aber meistens gleich.

Wenn maxLevel den Wert 0 hat, ist es der gleiche Algorithmus, aber ohne der Verwendung von Pyramiden (entspricht calcOpticalFlowLK). Pyramiden erlauben es, optische Flüsse für verschiedene Auflösungen des Bildes zu finden.

In [ ]:
# Parameter für Lucas Kanade Optischen Fluss
lk_params = dict( winSize  = (200,200),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))

In [ ]:
# Zeichne das Video auf
cap = cv2.VideoCapture(0)

# Nimm das erste Frame (Einzelbild) des Streams
ret, prev_frame = cap.read()

# Konvertiere es in Graustufen (Wir bezeichnen dies im folgenden als das vorherige Frame)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Finde die Ecken
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

# Erzeuge eine passende Maske des vorherigen Frames, um später darauf zu zeichnen
mask = np.zeros_like(prev_frame)


while True:
    
    # Nimm das derzeitige Frame
    ret,frame = cap.read()
    
    # Konvertiere es in Graustufen
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Berechne den optischen Fluss des Graustufenbildes
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # Verwende den zurückgegebenen Statusarray (die Statusausgabe)
    # Statusausgabe Statusvektor (chars ohne Vorzeichen); jedes Element des Vektors ist auf 1 gesetzt wenn
    # der Fluss der korrespondierenden Merkmale gefunden wurde, andernfalls ist es auf 0 gesetzt.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    # Verwende ravel, um Punkte zum Zeichnen von Linien und Kreisen zu erhalten
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        # Linien werden gezeichnet mit der vom ersten Frame erzeugten Maske
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 3)
        
        # Zeichne rote Kreise in Eckpunkte
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    
    # Zeige das Bild an, zusammen mit der Maske, auf die wir die Linie gezeichnet haben.
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Aktualisiere jetzt das vorherige Frame und vorherige Punkte
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()

# Dichter Optischer Fluss (Dense Optical Flow) in OpenCV

calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags) -> flow

Diese Funktion berechnet einen dichten Optischen Fluss mit dem Gunnar Farneback Algorithmus.

Hier sind die Parameter der Funktion und was sie repräsentieren:
   

* prev Erstes 8-bit Einkanal-Eingabebild.
* next Zweites Eingabebild mit gleicher Größe und gleichem Typ.
* flow Berechnetes Flussbild mit der selben Größe wie prev und dem Typ CV_32FC2.
* pyr_scale spezifiziert die Skalierung des Bildes (<1) für den Bau von Pyramiden für jedes Bild
  * pyr_scale=0.5 klassische Pyramide, bei der jede Ebene halb so groß ist wie die vorherige.
* levels Anzahl von Pyramidenebenen inklusive der initialen Ebene; levels=1 bedeutet, dass keine weiteren Ebenen erzeugt werden und nur die Originalbilder verwendet werden.
* winsize durchschnittliche Fenstergröße
    * größere Werte erhöhen die Robustheit des Algorithmus gegenüber Störsignalen im Bild und die Chance zur Erkennung schneller Bewegungen, erzeugen aber ein verschwommeneres Bewegungsfeld.
* iterations Anzahl von Iterationen, die der Algorithmus auf jeder Ebene der Pyramide durchführt.
* poly_n Verwendete Nachbarschaftsgröße für Pixel zum Finden der polynomialen Erweiterung jedes Pixels
    * größere Werte bedeuten, dass das Bild mit glatteren Oberflächen assoziiert wird, wodurch der Algorithmus robuster, aber das Bewegungsfeld auch verschwommener wird, normalerweise poly_n=5 oder 7.
* poly_sigma Standardabweichung der gauß'schen Funktion, die zur Glättung der Ableitungen als Basis für die polynomiale Erweiterung verwendet wird; für poly_n=5 kannst du poly_sigma=1.1 verwenden, für poly_n=7 wäre poly_sigma=1.5 ein guter Wert.

In [ ]:
import cv2 
import numpy as np

# Zeichne das Video auf
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

# Nimm die Graustufenversion des ersten Frames
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    # Wirf einen Blick in die obige Markdown-Zelle für eine Zusammenfassung dieser Parameter, die meisten sind nur die vorgeschlagenen Standardwerte
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    
    # Färbe die Kanäle basierend auf dem Winkel der Bewegung
    # Betrachte dein Video genau, die Flussrichtung der Bewegung bestimmt die Farbe!
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # Konvertiere zurück zu BGR mit imshow von OpenCV
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    # Setze das vorherige Bild auf das nächste Bild für die Schleife
    prvsImg = nextImg

    
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2

In [ ]:
cv2.cartToPolar